# Introdução

Convertendo a tabela do projeto Fapesp em seu equivalente fuzzy.

In [1]:
import pandas as pd
import numpy as np
from astropy.table import Table

In [2]:
def direct_fuzzy(Series, lower=.25, upper=.75, central=.5):
    inlist = Series.values
    lower_thresh = np.quantile(Series, lower)
    crossover = np.quantile(Series, central)
    upper_thresh = np.quantile(Series, upper)
    """Convert an interval or ratio-level variable into a fuzzy set by
    specifying the lower threshold, crossover point, and upper
    threshold.

    Ragin, Charles C. "Fuzzy Sets: Calibration Versus Measurement."
    Forthcoming in David Collier, Henry Brady, and Janet
    Box-Steffensmeir (eds.), Methodology volume of _Oxford Handbooks
    of Political Science_"""

    try:
        deviations   = [ x - crossover for x in inlist ]
        scalar_above = 3.0/(upper_thresh - crossover)
        scalar_below = -3.0/(lower_thresh - crossover)
        scalar_at    = 0

        log_odds = []
        for deviation in deviations:
            if deviation < 0:
                log_odds.append(deviation * scalar_below)
            elif deviation == 0: 
                log_odds.append(deviation * scalar_at)
            elif deviation > 0:
                log_odds.append(deviation * scalar_above)
            
        degree_membership = [ np.exp(x)/(1+np.exp(x)) for x in log_odds ]
        return degree_membership

    except: 
        TypeError: print("non-numeric input")
        raise

In [3]:
df = pd.DataFrame({
    "Países" : ["Alemanha", "Espanha", "França", "Holanda", "Itália", "Portugal"],
    "Maturidade Hipotecária": [30, 30, 19, 30, 22, 40],
    "Taxa de juros hipotecária": ["Fixa", "Variável", "Fixa", "Fixa", "Variável", "Variável"],
    "Reembolso antecipado" : ["C/L", "C/L", "C/L", "C", "L", "L"],
    "Retirada de capital próprio": ["Não permitido", "Limitado", "Não permitido", "Permitido", "Não permitido", "Sem informação"],
    "Financiamento pelo mercado de capitais": [.14, .45, .12, .25, .20, .27],
    "Execução Hipotecária (meses)": [9, 8, 20, 5, 56, 24]
})
df.set_index("Países", inplace=True)
df["Execução Hipotecária (meses)"] = direct_fuzzy(df["Execução Hipotecária (meses)"])
df["Maturidade Hipotecária"] = direct_fuzzy(df["Maturidade Hipotecária"]); df.loc["Portugal", "Maturidade Hipotecária"] = 1.0;
#df["Taxa de juros hipotecária"] = df["Taxa de juros hipotecária"].replace("Fixa", 0); df["Taxa de juros hipotecária"] = df["Taxa de juros hipotecária"].replace("Variável", 1)
df["Taxa de juros hipotecária"] = df["Taxa de juros hipotecária"].map({"Variável":1.0, "Fixa":0.0})
df["Reembolso antecipado"] = df["Reembolso antecipado"].map({"C": 0.0, "L":1.0, "C/L":0.5})
df["Retirada de capital próprio"] = df["Retirada de capital próprio"].map({"Permitido":1.0, "Não permitido":0.0, "Limitado":0.5, "Sem informação":np.nan})
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars


,Maturidade Hipotecária,Taxa de juros hipotecária,Reembolso antecipado,Retirada de capital próprio,Financiamento pelo mercado de capitais,Execução Hipotecária (meses)
Países,,,,,,
Alemanha,0.500000,0.0,0.5,0.0,0.14,0.066608
Espanha,0.500000,1.0,0.5,0.5,0.45,0.042290
França,0.004070,0.0,0.5,0.0,0.12,0.874481
Holanda,0.500000,0.0,0.0,1.0,0.25,0.010354
Itália,0.017986,1.0,1.0,0.0,0.20,1.000000
Portugal,1.000000,1.0,1.0,NaN,0.27,0.966201


# Importando hipotecarização

In [4]:
countries =[
    "DEU",
    "ESP",
    "FRA",
    "NLD",
    "ITA",
    "PRT",
    "USA"
]

In [5]:
jorda = pd.read_stata('http://www.macrohistory.net/JST/JSTdatasetR4.dta', )
jorda.set_index(["iso", "year"], inplace=True, )
jorda = jorda.loc[countries,["tmort", "tloans"]]
jorda["Hipotecarização"] = jorda["tmort"]/jorda["tloans"]
jorda = jorda.reset_index("year")
jorda

,year,tmort,tloans,Hipotecarização
iso,,,,
FRA,1870.0,0.0087,NaN,NaN
FRA,1871.0,0.0088,NaN,NaN
FRA,1872.0,0.0089,NaN,NaN
FRA,1873.0,0.0085,NaN,NaN
FRA,1874.0,0.0085,NaN,NaN
...,...,...,...,...
USA,2012.0,6141.2600,9444.211,0.650267
USA,2013.0,6255.0000,9730.200,0.642844
USA,2014.0,6461.2000,10273.200,0.628937


In [6]:
{pais: jorda.loc[pais, "Hipotecarização"].mean() for pais in countries}

{'DEU': 0.4109952150232498,
 'ESP': 0.23619307348868787,
 'FRA': 0.3187597846659342,
 'NLD': 0.42690076362611973,
 'ITA': 0.254989916608997,
 'PRT': 0.208389803988441,
 'USA': 0.4874863682351805}

In [7]:
df["Hipotecarização média (19870-2016)"] = [jorda.loc[pais, "Hipotecarização"].mean() for pais in countries if pais is not "USA"]
df

,Maturidade Hipotecária,Taxa de juros hipotecária,Reembolso antecipado,Retirada de capital próprio,Financiamento pelo mercado de capitais,Execução Hipotecária (meses),Hipotecarização média (19870-2016)
Países,,,,,,,
Alemanha,0.500000,0.0,0.5,0.0,0.14,0.066608,0.410995
Espanha,0.500000,1.0,0.5,0.5,0.45,0.042290,0.236193
França,0.004070,0.0,0.5,0.0,0.12,0.874481,0.318760
Holanda,0.500000,0.0,0.0,1.0,0.25,0.010354,0.426901
Itália,0.017986,1.0,1.0,0.0,0.20,1.000000,0.254990
Portugal,1.000000,1.0,1.0,NaN,0.27,0.966201,0.208390


In [11]:
df.to_latex(
    "../tabs/fuzzyfied.tex", 
    caption="Especificidades institucionais fuzzificadas",
    decimal=",", label="tab::fuzzyficada", 
    encoding="utf-8", column_format="cccccccc",
    float_format="%.3f", na_rep="-", header=False, index_names=False
)